In [149]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

**old**: doesn't work

In [ ]:
from z3 import *
import numpy as np
from encodings_utils import *

# Example function to create the Z3 constraints for MCP
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # A courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # 1. Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]

    # CONSTRAINTS
    # 1. Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[c]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, min(l[c], max(s)), f"courier_{c}_load"))

    # 2. Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times

    # 3. Each courier cannot be in two places at the same time
    for c in range(m):
        for i in range(max_times + 1):
            solver.add(exactly_one_bw([v[c][i][t] for i in range(n + 1)], f"courier_{c}_time_{t}"))
            # let's use the bitwise encoding since it's more scalable for a large number of variables..
            # ..due to its logarithmic growth in auxiliary variables and constraints.

    # 4. Each item i is delivered exactly once
    for i in range(n):
        solver.add(exactly_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)], f"exactly_once_{i}"))

    # 5. If each courier must deliver at least one package
    for c in range(m):
        solver.add(Or([v[c][i][t] for i in range(n)]))

    return solver, v

# Example function to add distance constraint
def add_distance_constraint_standard(solver, instance, v, upperBound):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]  # max_times horizon
    distances = instance["D"]  # Distances between packages

    for c in range(m):
        dists = []
        for start in range(n + 1):
            for end in range(n + 1):
                for _ in range(distances[start][end]):
                    dists.append(Or(Not(v[c][start][end]), BoolVal(True)))
        solver.add(at_most_k_seq(dists, upperBound, f"courier_dist_{c}_{upperBound}"))

        print(f"Courier {c}, dists size: {len(dists)}")

    # Ensure that the number of nodes is not exceeded
    if np.sum(distances[:max_times]) <= upperBound:
        for c in range(m):
            solver.add(v[c][n][max_times + 1])

# Example function to solve the MCP using Z3
def run_sat_linear_search(instance):
    # Copy of the instance since we will modify it
    instance = dict(instance)

    # Initialize the Z3 solver and variables
    solver, v = mcp(instance)

    # Set the original upper bound and lower bound for linear search
    original_upper_bound = instance["max_dist"]
    lower_bound = instance["min_dist"]
    upper_bound = original_upper_bound

    # Step size for the linear search (can be adjusted)
    step = 10

    # Linear search for finding a feasible solution
    distance_limit = lower_bound
    while distance_limit <= upper_bound:
        solver.push()

        # Add the distance constraint to the solver
        add_distance_constraint_standard(solver, instance, v, distance_limit)

        # Check if the problem is satisfiable
        if solver.check() == sat:
            model = solver.model()
            solution = extract_solution(model, v, instance["m"], instance["n"], instance["max_times"])
            return solution, objective_function(solution, instance)

        # Increment the distance limit for the next iteration
        distance_limit += step

        # Pop the constraint for the next iteration
        solver.pop()

    # Return unsat if no solution found within the distance constraints
    return "unsat", None

# Example function to extract the solution from Z3 model
def extract_solution(model, v, m, n, max_times):
    solution = []

    for c in range(m):
        courier_solution = []
        for t in range(1, max_times + 1):
            for i in range(n):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)
        solution.append(courier_solution)

    return solution

# Example objective function: total distance traveled by all couriers
def objective_function(solution, instance):
    total_distance = 0
    distances = instance["D"]
    n = instance["n"]

    for courier_solution in solution:
        if not courier_solution:
            continue
        depot = n  # Starting node
        for node in courier_solution:
            total_distance += distances[depot][node]
            depot = node
        total_distance += distances[depot][n]  # Return to starting node

    return total_distance

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Example instance data
instance = {
    "m": 3,  # Number of couriers
    "n": 4,  # Number of packages
    "l": [15, 20, 25],  # Weights each courier can carry
    "s": [5, 8, 7, 6],  # Sizes of packages
    "D": [
        [0, 10, 20, 15, 10],
        [10, 0, 25, 30, 20],
        [20, 25, 0, 35, 15],
        [15, 30, 35, 0, 25],
        [10, 20, 15, 25, 0],
    ],  # Distances between packages
}

# Compute additional info based on the instance
additional_info = compute_additional_info(instance)
instance.update(additional_info)

# Run the linear search SAT solver
result, obj_value = run_sat_linear_search(instance)

# Process the final result
if result == "unsat":
    print("No solution found within the distance constraints.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx + 1}: {path}")

    print(f"Objective Function Value: {obj_value}")


**new**: testing without distance computation

In [115]:
from z3 import *
import numpy as np
from utils import *

# Example function to create the Z3 constraints for MCP
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # 1. Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    # 1. Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2. Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3. Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4. Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))
    
    # 5. All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))
    
    return solver, v

# Instance 05
instance = {
    "m": 2,  # Number of couriers
    "n": 3,  # Number of packages
    "l": [18, 30],  # Weights each courier can carry
    "s": [20, 17, 6],  # Sizes of packages
    "D": [
        [0, 21, 86, 99],
        [21, 0, 71, 80],
        [92, 71, 0, 61],
        [59, 80, 61, 0],
    ],  # Distances between packages
}

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

instance.update(additional_info)
solver, v = mcp(instance=instance)

if solver.check() == sat:
    model = solver.model()
    #print(model)
    solution = []
    for c in range(instance["m"]):
        courier_solution = []
        for t in range(instance["max_times"]+1):
            for i in range(instance["n"]+1):
                if model[v[c][i][t]] == True:
                    courier_solution.append(i + 1)
        solution.append(courier_solution)
    print(solution)
else: print("unsat")

[[4, 2, 4], [4, 1, 3, 4]]


**new**: testing

In [152]:
from z3 import *
import numpy as np
from utils import *

# Example function to create the Z3 constraints for MCP
def mcp(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Items' sizes
    max_times = instance["max_times"]  # Time horizon: a courier can carry at most max_times items..
    # .. and since at each timestep a courier can pick exactly one item, max_times refers to the max time steps possible

    solver = Solver()
    solver.set("timeout", timeout)

    # VARIABLES
    # - Represent that each courier c picks the item i at the time step t (add 1 for depot)
    v = [[[Bool(f"x_{c}_{i}_{t}") for t in range(max_times + 1)] for i in range(n + 1)] for c in range(m)]
    
    # CONSTRAINTS
    # 1. Each courier c can carry at most l[c] kg
    for c in range(m):
        weight_set = []
        for i in range(n):
            for t in range(1, max_times):
                for _ in range(s[i]):
                    weight_set.append(v[c][i][t])
        solver.add(at_most_k_seq(weight_set, l[c], f"courier_{c}_load"))
        
    # 2. Each courier c starts and ends at position i = n
    for c in range(m):
        solver.add(v[c][n][0] == True)  # Start at position n at time 0
        solver.add(v[c][n][max_times] == True)  # End at position n at max_times
    
    # 3. Each courier must deliver at least one item
    for c in range(m):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for i in range(n)]))
    
    # 4. Each courier cannot pick the same item more than once
    for c in range(m):
        for i in range(n):
            solver.add(at_most_one_bw([v[c][i][t] for t in range(1, max_times)], f"exactly_once_courier{c}"))
    
    # 5. All items should be picked up
    for i in range(n):
        solver.add(at_least_one_bw([v[c][i][t] for t in range(1, max_times) for c in range(m)]))

    # 6. Distance computation
    upper_bound = instance["max_dist"]  # Maximum allowed distance

    # Flattened list of Boolean variables representing distance usage
    all_distance_vars = []
    distance_vars = {}

    # Create distance variables and constraints
    for c in range(m):
        for t in range(max_times):
            for start in range(n + 1):
                for end in range(n + 1):
                    if start != end:
                        distance_var = Bool(f"dist_{c}_{start}_{end}_{t}")
                        all_distance_vars.append(distance_var)
                        distance_vars[(c, start, end, t)] = distance_var

                        # Define constraints for distance variables
                        if start < n and end < n:  # Ensure valid indices
                            travel_start = v[c][start][t]
                            travel_end = v[c][end][t + 1]
                            
                            # Encode: distance_var -> (travel_start and travel_end)
                            solver.add(Or(Not(distance_var), travel_start))
                            solver.add(Or(Not(distance_var), travel_end))
                            
                            # Encode: (travel_start and travel_end) -> distance_var
                            solver.add(Or(Not(travel_start), Not(travel_end), distance_var))
    
    # Use at_most_k_seq to ensure the total distance does not exceed upper_bound
    solver.add(at_most_k_seq(all_distance_vars, upper_bound, "distance_limit"))

    # return model and the updated 3D variable
    return solver, v

# Define a function to extract the solution and compute the total distance for each courier
def compute_total_distance(model, v, instance):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    distances = instance["D"]  # Distance matrix
    max_times = instance["max_times"]  # Maximum number of time steps
    
    total_distances = [0] * m  # Initialize a list to store the total distance for each courier

    for c in range(m):
        courier_path = []
        current_location = n  # Start at the depot
        courier_distance = 0

        # Collect the path
        for t in range(1, max_times + 1):
            for i in range(n):
                if model[v[c][i][t]]:
                    next_location = i
                    if next_location != current_location:
                        distance = distances[current_location][next_location]
                        courier_distance += distance
                    courier_path.append(next_location)
                    current_location = next_location                    
        
        # Add distance to return to the depot at the end
        return_distance = distances[current_location][n]
        courier_distance += return_distance        
        total_distances[c] = courier_distance

    return total_distances

# Example usage
def print_total_distances(instance, model, v, solution):    
    # Compute the total distance for each courier
    total_distances = compute_total_distance(model, v, instance)
    
    # Print the total distance for each courier
    for c in range(instance["m"]):
        print(f"Courier {c + 1} path: {solution[c]}")
        print(f"Courier {c + 1} total distance: {total_distances[c]}")

def extract_solution(model, instance):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of packages
    max_times = instance["max_times"]
    solution = []
    for c in range(m):
        courier_solution = []
        for t in range(max_times + 1):
            for i in range(n + 1):
                if model[v[c][i][t]]:
                    courier_solution.append(i + 1)  # Store 1-based index for better readability
        solution.append(courier_solution)
    return solution

# Example function to compute additional info for the instance
def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]

    # Number of max_times steps
    max_times = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load (max among all sizes)
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    additional_info = {
        "max_times": max_times,
        "max_dist": max_distance,
        "min_dist": 0,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Importing instance
instance_num = "05"
file_path = os.path.join('Instances', f'inst{instance_num}.dat')
instance = read_dat_file(file_path)
instance.update(additional_info)

# Running the model
solver, v = mcp(instance=instance)

# Check satisfiability
if solver.check() == sat:
    model = solver.model()
    solution = extract_solution(model, instance)
    #print(solution)
    print_total_distances(instance, model, v, solution)
else: print("unsat")

c:\Users\follo\OneDrive\Desktop\uniBo\corsi\Optimization\CDMO-project\project\SAT
Courier 1 path: [4, 2, 4]
Courier 1 total distance: 160
Courier 2 path: [4, 1, 3, 4]
Courier 2 total distance: 206


## binary search

In [18]:
from z3 import *
from utils2 import *
import numpy as np

def mcp_st(instance, timeout=3000000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Sizes of items
    max_nodes = instance["max_nodes"]  # max_nodes horizon
    min_load = instance["min_load"]  # Minimum load each courier can carry
    max_load = instance["max_load"]  # Maximum load each courier can carry
    equal_load_matrix = instance["equal_matrix"]

    solver = Solver()
    solver.set("timeout", timeout)

    # Variables
    b_path = [[[Bool(f"x_{i}_{j}_{k}") for k in range(max_nodes + 1)] for j in range(n + 1)] for i in range(m)]

    path = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n + 1)]
          for start in range(n + 1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range(m):
        loads = []
        for j in range(n):
            for k in range(1, max_nodes):
                for _ in range(s[j]):
                    loads.append(b_path[i][j][k])
        solver.add(at_most_k_seq(loads, min(l[i], max_load), f"courier_{i}_load"))

    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(b_path[i][n][0])  # Start at position n at max_nodes 0
        solver.add(b_path[i][n][max_nodes])  # End at position n at max_nodes

    # 3. Each courier cannot be in two places at the same time
    for i in range(m):
        for k in range(max_nodes + 1):
            solver.add(exactly_one_bw([b_path[i][j][k] for j in range(n + 1)], f"item_{i}_{k}"))

    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([b_path[i][j][k] for k in range(1, max_nodes) for i in range(m)], f"exactly_once_{j}"))

    # 5. If each courier must deliver at least one package
    for i in range(m):
        solver.add(Or([b_path[i][j][1] for j in range(n)]))

    return solver, b_path, path

def add_distance_constraint_standard(solver, instance, v, d, upperbound):
    m = instance["m"]  # couriers
    n = instance["n"]  # items
    max_nodes = instance["max_nodes"]  # max_nodes
    distances = instance["D"]  # distances between items
    for i in range(m):
        dists = []
        for start in range(n + 1):
            for end in range(n + 1):
                for z in range(distances[start][end]):
                    dists.append(d[i][start][end])
        solver.add(at_most_k_seq(dists, upperbound, f"courier_dist_{i}_{upperbound}"))
    
    dist_flat = np.array(instance['D']).flatten()
    dist_flat = dist_flat[dist_flat != 0]
    dist_flat = np.sort(dist_flat)

    max_k = 0

    while np.sum(dist_flat[:max_k]) <= upperbound and max_k < n:
        max_k += 1
        
    if max_k < max_nodes:
        for i in range(m):
            solver.add(v[i][n][max_k + 1])

def run_sat(instance):
    # copy of the instance since we will modify it
    instance = dict(instance)
    mcp = mcp_st
    add_constraint = add_distance_constraint_standard

    # add additional info to the instance
    add_additional_info(instance)
    # original upper bound -> used in the binary search to check if the problem is unsat
    original_upper_bound = instance["max_dist"]
    # lower bound for the binary search
    lower_bound = instance["min_dist"]
    # upper bound for the binary search
    upper_bound = original_upper_bound

    # pivot for the binary search
    pivot = (original_upper_bound + lower_bound) // 2

    # solver creation based on the model used
    solver, v, d = mcp(instance) 

    res = None

    best_solution = None

    # binary search using bounds
    while True:
        # if the lower bound is equal to the upper bound, we have found the optimal solution
        if lower_bound >= upper_bound and res is not None:
            return best_solution, objective_function(best_solution, instance)

        # push/pop mechanism for adding new constraint and solving the problem
        solver.push()

        # add the constraint to the solver to force the distance to be lower or equal than the pivot
        add_constraint(solver, instance, v, d, pivot)

        # solve the problem with the current constraints
        res = solve_mcp(solver)

        # if the problem is unsat
        if res == unsat:
            # if we never found a solution then upper bound is the same as the original upper bound hence the problem is unsat
            if upper_bound == original_upper_bound:
                # if the upper bound is equal to the original upper bound, the problem is unsat since we never found a solution
                return "unsat", None
            
            # updating lower
            lower_bound = pivot + 1
            # pop the constraint since we will add a new one
            solver.pop()
            # update the pivot
            pivot = (upper_bound + lower_bound) // 2
        else:
            # if the problem is sat, we save the solution
            best_solution = format_solution(instance['m'], instance['n'], instance['max_nodes'], res, v)
            upper_bound = pivot
            pivot = (upper_bound + lower_bound) // 2


def add_additional_info(instance):
    m = instance["m"]
    l = instance["l"]
    equal_matrix = np.full((m, m), BoolVal(False))
    for i in range(m):
        for i1 in range(m):
            if l[i] == l[i1]:
                equal_matrix[i, i1] = BoolVal(True)
    instance["equal_matrix"] = equal_matrix

def solve_mcp(solver):
    if solver.check() == sat:
        return solver.model()
    else:  
        return unsat

def format_solution(m, n, max_nodes, model, v):
    solution = []

    for i in range(m):
        courier_solution = []

        for k in range(max_nodes):
            for j in range(n):
                if model[v[i][j][k]]:
                    courier_solution.append(j + 1)

        solution.append(courier_solution)

    return solution

def objective_function(solution, instance):
    # Example objective function: total distance traveled by all couriers
    total_distance = 0
    distances = instance["D"]
    n = instance["n"]

    for courier_solution in solution:
        if not courier_solution:
            continue
        depot = n  # Starting node
        for node in courier_solution:
            total_distance += distances[depot][node]
            depot = node
        total_distance += distances[depot][n]  # Return to starting node

    return total_distance

def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]
    
    # Number of max_nodes steps
    max_nodes = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load 
    #max_load = np.sum(np.sort(instance["s"])[-max_nodes:])
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    min_distance = 0

    #print(max_nodes, min_load, max_load, min_distance, max_distance)

    additional_info = {
        "max_nodes": max_nodes,
        "max_dist": max_distance,
        "min_dist": min_distance,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

#################################################################################

# choose inst.
NUM_INST = 5
instance = instances[NUM_INST - 1]

# Compute additional info based on the instance
additional_info = compute_additional_info(instance)
instance.update(additional_info)

# Run the SAT solver
result, obj_value = run_sat(instance)

# Process the final result
if result == "unsat":
    print("No solution found.")
else:
    print("Best Solution Found:")
    for courier_idx, path in enumerate(result):
        print(f"Courier {courier_idx}: {path}")

    print(f"Objective Function Value: {obj_value}")

No solution found.


In [10]:
import time
from z3 import *
from utils2 import *
import numpy as np

def mcp_st(instance, timeout=50000):
    m = instance["m"]  # Number of couriers
    n = instance["n"]  # Number of items
    l = instance["l"]  # Weights each courier can carry
    s = instance["s"]  # Sizes of items
    max_nodes = instance["max_nodes"]  # max_nodes horizon
    min_load = instance["min_load"]  # Minimum load each courier can carry
    max_load = instance["max_load"]  # Maximum load each courier can carry
    equal_load_matrix = instance["equal_matrix"]

    solver = Solver()
    solver.set("timeout", timeout)

    # Variables
    b_path = [[[Bool(f"x_{i}_{j}_{k}") for k in range(max_nodes + 1)] for j in range(n + 1)] for i in range(m)]
    path = [[[Bool(f"d_{i}_{start}_{end}") for end in range(n + 1)]
             for start in range(n + 1)] for i in range(m)]

    # Constraints
    # 1. Each courier can carry at most l[i] kg
    for i in range(m):
        loads = []
        for j in range(n):
            for k in range(1, max_nodes):
                for _ in range(s[j]):
                    loads.append(b_path[i][j][k])
        solver.add(at_most_k_seq(loads, min(l[i], max_load), f"courier_{i}_load"))

    # 2. Each courier i starts and ends at position j = n
    for i in range(m):
        solver.add(b_path[i][n][0])  # Start at position n at max_nodes 0
        solver.add(b_path[i][n][max_nodes])  # End at position n at max_nodes

    # 3. Each courier cannot be in two places at the same time
    for i in range(m):
        for k in range(max_nodes + 1):
            solver.add(exactly_one_bw([b_path[i][j][k] for j in range(n + 1)], f"item_{i}_{k}"))

    # 4. Each package j is delivered exactly once
    for j in range(n):
        solver.add(exactly_one_bw([b_path[i][j][k] for k in range(1, max_nodes) for i in range(m)], f"exactly_once_{j}"))

    # 5. Each courier must deliver at least one package
    for i in range(m):
        solver.add(Or([b_path[i][j][1] for j in range(n)]))

    return solver, b_path, path

def add_distance_constraint_standard(solver, instance, v, d, upperbound):
    m = instance["m"]  # couriers
    n = instance["n"]  # items
    max_nodes = instance["max_nodes"]  # max_nodes
    distances = instance["D"]  # distances between items
    for i in range(m):
        dists = []
        for start in range(n + 1):
            for end in range(n + 1):
                for z in range(distances[start][end]):
                    dists.append(d[i][start][end])
        solver.add(at_most_k_seq(dists, upperbound, f"courier_dist_{i}_{upperbound}"))
    
    dist_flat = np.array(instance['D']).flatten()
    dist_flat = dist_flat[dist_flat != 0]
    dist_flat = np.sort(dist_flat)

    max_k = 0

    while np.sum(dist_flat[:max_k]) <= upperbound and max_k < n:
        max_k += 1
        
    if max_k < max_nodes:
        for i in range(m):
            solver.add(v[i][n][max_k + 1])

def run_sat(instance):
    # Copy of the instance since we will modify it
    instance = dict(instance)
    mcp = mcp_st
    add_constraint = add_distance_constraint_standard

    # Add additional info to the instance
    add_additional_info(instance)
    # Original upper bound -> used in the search to check if the problem is unsat
    original_upper_bound = instance["max_dist"]
    # Lower bound for the search
    lower_bound = instance["min_dist"]
    # Upper bound for the search (start from the lower bound)
    upper_bound = original_upper_bound

    # Pivot for the search
    pivot = (original_upper_bound + lower_bound) // 2

    # Solver creation based on the model used
    solver, v, d = mcp(instance)

    res = None
    best_solution = None

    start_time = time.time()

    # Search using bounds
    while True:
        # Check for timeout
        elapsed_time = time.time() - start_time
        if elapsed_time > 300:
            print("Timeout reached.")
            break

        # If the lower bound is equal to the upper bound, we have found the optimal solution
        if lower_bound >= upper_bound and res is not None:
            return best_solution, objective_function(best_solution, instance)

        # Push/pop mechanism for adding new constraint and solving the problem
        solver.push()

        # Add the constraint to the solver to force the distance to be lower or equal than the pivot
        add_constraint(solver, instance, v, d, pivot)

        # Solve the problem with the current constraints
        res = solve_mcp(solver)

        # If the problem is unsat
        if res == unsat:
            # If we never found a solution then upper bound is the same as the original upper bound hence the problem is unsat
            if upper_bound == original_upper_bound:
                # If the upper bound is equal to the original upper bound, the problem is unsat since we never found a solution
                return "unsat", None

            # Updating lower
            lower_bound = pivot + 1
            # Pop the constraint since we will add a new one
            solver.pop()
            # Update the pivot
            pivot = (upper_bound + lower_bound) // 2
        else:
            # If the problem is sat, we save the solution
            best_solution = format_solution(instance['m'], instance['n'], instance['max_nodes'], res, v)
            upper_bound = pivot
            pivot = (upper_bound + lower_bound) // 2

def add_additional_info(instance):
    m = instance["m"]
    l = instance["l"]
    equal_matrix = np.full((m, m), BoolVal(False))
    for i in range(m):
        for i1 in range(m):
            if l[i] == l[i1]:
                equal_matrix[i, i1] = BoolVal(True)
    instance["equal_matrix"] = equal_matrix

def solve_mcp(solver):
    if solver.check() == sat:
        return solver.model()
    else:  
        return unsat

def format_solution(m, n, max_nodes, model, v):
    solution = []

    for i in range(m):
        courier_solution = []

        for k in range(max_nodes):
            for j in range(n):
                if model[v[i][j][k]]:
                    courier_solution.append(j + 1)

        solution.append(courier_solution)

    return solution

def objective_function(solution, instance):
    # Example objective function: total distance traveled by all couriers
    total_distance = 0
    distances = instance["D"]
    n = instance["n"]

    for courier_solution in solution:
        if not courier_solution:
            continue
        depot = n  # Starting node
        for node in courier_solution:
            total_distance += distances[depot][node]
            depot = node
        total_distance += distances[depot][n]  # Return to starting node

    return total_distance

def compute_additional_info(instance):
    n = instance["n"]
    m = instance["m"]
    D = instance["D"]
    
    # Number of max_nodes steps
    max_nodes = (n // m) + 3

    # Minimum load (min among all sizes)
    min_load = min(instance["s"])

    # Max load 
    max_load = max(instance["s"])

    # Maximum distance
    max_distance = np.sum(np.max(D, axis=1))

    min_distance = 0

    additional_info = {
        "max_nodes": max_nodes,
        "max_dist": max_distance,
        "min_dist": min_distance,
        "min_load": min_load,
        "max_load": max_load,
    }

    return additional_info

# Choose instance.
NUM_INST = 2
instance = instances[NUM_INST - 1]

# Compute additional info based on the instance
additional_info = compute_additional_info(instance)
instance.update(additional_info)

# Run the SAT solver with a 5-minute timeout
try:
    result, obj_value = run_sat(instance)
    # Process the final result
    if result == "unsat":
        print("No solution found.")
    else:
        print("Best Solution Found:")
        for courier_idx, path in enumerate(result):
            print(f"Courier {courier_idx}: {path}")

        print(f"Objective Function Value: {obj_value}")

except TimeoutException:
    print("Timeout reached. No optimal solution found within the time limit.")


Exception ignored in: <function AstRef.__del__ at 0x0000013E3A9F55A0>
Traceback (most recent call last):
  File "c:\Users\follo\AppData\Local\Programs\Python\Python310\lib\site-packages\z3\z3.py", line 352, in __del__
    Z3_dec_ref(self.ctx.ref(), self.as_ast())
  File "c:\Users\follo\AppData\Local\Programs\Python\Python310\lib\site-packages\z3\z3core.py", line 1626, in Z3_dec_ref
    _elems.f(a0, a1)
KeyboardInterrupt: 


In [1]:
import os

def read_dat_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Read m and n
    m = int(lines[0].strip())
    n = int(lines[1].strip())
    
    # Read l vector
    l = list(map(int, lines[2].strip().split()))
    
    # Read s vector
    s = list(map(int, lines[3].strip().split()))
    
    # Read D matrix
    D = []
    for line in lines[4:]:
        D.append(list(map(int, line.strip().split())))
    
    return {
        'm': m,
        'n': n,
        'l': l,
        's': s,
        'D': D
    }

def read_all_dat_files(directory):
    instances = []
    for filename in os.listdir(directory):
        if filename.endswith('.dat'):
            file_path = os.path.join(directory, filename)
            instance = read_dat_file(file_path)
            instances.append(instance)
    return instances

# Directory containing .dat files
directory = 'instances'

# Read all .dat files and populate instances
instances = read_all_dat_files(directory)